In [1]:
#pip install pytube3
#pip install face_recognition
from google.colab import drive
drive.mount('/content/drive')
#pip freeze --local > /content/drive/My\ Drive/colab_installed.txt
#pip install -r /content/drive/My\ Drive/colab_installed.txt


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -q pytube3
!pip install -q face_recognition
%cd drive/'My Drive'/Programa

     |████████████████████████████████| 100.2MB 42kB/s 
/content/drive/My Drive/Programa


In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 23 12:18:03 2020

@author: Tsuru
"""
import numpy as np
import sys
import face_recognition #Librería de reconocimiento facial
import os #Librería para interactuar con el entorno
import cv2 #Librería de visión artificial
from sklearn.neural_network import MLPClassifier #Clasificador MLP
from sklearn.externals import joblib #Para guardar/cargar clases en archivos
import pandas as pd
from pytube import YouTube
from google.colab.patches import cv2_imshow


#%%
################################# Funcion 1 #################################

#Función para descargar videos de el archivo Listado.csv

def download_video_tool():
    
    try:
        download_video_list=pd.read_csv('Listado.csv', sep=';')
    except Exception:
        print("Listado.csv no encontrado en la carpeta actual",os.getcwd())
        
    downloaded_videos=0
    total_videos=len(download_video_list.Enlace)

    if(total_videos == 0):
        print("No hay videos en lista. Agrega enlaces a Listado.csv")

    elif(total_videos == 1):
        download_stream=YouTube(download_video_list.iloc[1,1])
        print("Descargando",download_stream.title)
        download_stream.streams.get_by_itag(18).download(output_path='Descargados')
        print("Descarga finalizada")
    
    else:    
        print("Iniciando descarga: se van a descargar",total_videos,"videos")
        for dl in download_video_list.iloc[:,1]:
            download_stream=YouTube(dl)
            #print(download_stream.streams.filter(file_extension='mp4'))
            print("Descargando:",download_stream.title)
            download_stream.streams.get_by_itag(18).download(output_path='Descargados')
            downloaded_videos += 1
            print("Descarga finalizada:", downloaded_videos,"de", total_videos)


#%%
################################# Funcion 1 #################################
#Función que crea el modelo MLP
def model_creator():
    #Por defecto, se tomará la carpeta Source
    dirs=os.listdir(path='./Personas')
    
    #Inicializando parámetros
    encodings_final=[]
    tags=[]
    known_names=[]
    
    print("Analizando directorios:")
    for (directorio,index) in zip(dirs, range(len(dirs))):
        #Muestra el directorio analizado
        print(str(index+1) + ') ' + directorio)
        load_dir(directorio, encodings_final, index, tags)
        #Suponiendo que hay al menos un rostro correcto en cada directorio
        known_names.append(directorio)
    
    #Si se han obtenido algunas etiquetas (y por tanto encodings)
    if tags:
        #Crea y entrena el clasificador
        clf = MLPClassifier(verbose=False, max_iter=1000, activation= 'tanh',
        hidden_layer_sizes = (100))
        model = clf.fit(encodings_final, tags)
        #Guarda el modelo de MLP entrenado y la lista de nombres
        joblib.dump(model, './Model/MLPClassifier.pkl')
        joblib.dump(known_names, './Model/Names.pkl')
        print("Clasificador MLP creado.")
    else:
        print("No se obtuvieron rostros")
        

#%%            
################################# Funcion 2 #################################

def load_dir(directorio, encodings_final, index, tags):
    path= './Personas/' + directorio +'/'
    #Lista con los nombres de las imágenes de la carpeta de la persona
    file_list = os.listdir(path)
    for imagen in file_list:
    #Forma la ruta de la imagen
        complete_path= path + imagen
    #Carga la imagen y extrae su codificación (encodings)
        imagen = face_recognition.load_image_file(complete_path)
        encodings=face_recognition.face_encodings(imagen)
    #Si se ha obtenido, se añade a la lista final de encodings y de etiquetas
        if encodings:
            encodings_final.append(encodings[0])
            tags.append(index)

#Función que analiza los rostros de una imagen
def image(image_input):
  #Toma la ubicación y encoding de cada rostro de la imagen
  image = face_recognition.load_image_file(image_input)
  (detected_names, face_locations)=detect(image)
  #Inserta la ventana de identificación en la imagen
  insert_name(detected_names, face_locations, image)

  #Pasa del formato de color de face_recognition (RGB) al de OpenCV (BGR)
  image = image[:, :, ::-1]

  #Guarda la imagen procesada
  splited_name=os.path.splitext(image_input)
  name_image="./Output/"+splited_name[0]+"_out"+splited_name[1]
  cv2_imshow(image)
  cv2.imwrite(name_image, image)
  print(name_image+" creada.")
            

#%%
def detect(rgb_frame):

    try:
        #Carga el modelo
        model=joblib.load('./Model/MLPClassifier.pkl')
        known_names=joblib.load('./Model/Names.pkl')
    except Exception:
        sys.exit("No se pudo cargar el clasificador.")

    detected_names=[]
    #Establece el umbral: mínimo parecido posible
    prox_min=0.8

    # Encuentra todos los rostros del frame y obtiene sus encodings
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame)

    n_face=0
    for face_encoding in face_encodings:
        [proximities]=model.predict_proba(face_encoding.reshape(1,-1))

        prox_max=max(proximities)
        if prox_max >=prox_min:
            tag=np.argmax(proximities)
            proximity = prox_max*100
            name = known_names[tag] + '('+str(round(proximity,2))+'%)'
            detected_names.append(name)
        else:
            name = "Unknown"
            detected_names.append(name)
            n_face+=1

    return detected_names, face_locations

#%%
#Función que añade a una imagen el marco con la identificación de cada rostro detectado
def insert_name(names, face_locations, frame):
    for (top, right, bottom, left), name in zip(face_locations, names):
        #Añade el marco y el texto a la imagen
        cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
        cv2.putText(frame, name, (left, top-6), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0), 1)

#%%
def video(video_input):
#Si video_input es 0, se usa la webcam y se muestra por pantalla


 #Abre el vídeo
 video_capture = cv2.VideoCapture(video_input)
 if (video_capture.isOpened() == False):
     print("No se pudo leer el video.")
 else:
 #Si se ha tomado un archivo de vídeo
    if(video_input):
        # Obtiene la resolución del video en formato entero.
        frame_width = int(video_capture.get(3))
        frame_height = int(video_capture.get(4))

        splited_name=os.path.splitext(video_input)
        video_name="./Output/"+splited_name[0]+"_out.avi"
        # Crea el objeto que se encargará de guardar el vídeo destino
        out = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'XVID'),video_capture.get(cv2.CAP_PROP_FPS), (frame_width,frame_height))
        print("Procesando video en: "+video_name+"...")
    
    else:
        print("Pulse ESPACIO para detener la grabación.")

#Se procesará un fotograma sí y otro no
    process_this_frame=False
    while video_capture.isOpened():
            # Toma un frame y comprueba
        ret, frame = video_capture.read()
        process_this_frame=not process_this_frame
        

        if ret==True:
            if(process_this_frame):
              #Convierte la imagen de color BGR (OpenCV) a RGB (Face_Recognition)
              rgb_frame = frame[:, :, ::-1]
              print(rgb_frame.shape)
              (detected_names, face_locations)=detect(rgb_frame)
              insert_name(detected_names, face_locations, frame)
            #Si se toma de la cámara, se muestra por pantalla
            if(not video_input):
              # Muestra la iamgen por pantalla
              cv2.imshow('Video', frame)
            else:
              out.write(frame)
               # Se detiene si se presiona la barra espaciadora
            if cv2.waitKey(1) & 0xFF == ord(' '):
              video.release()
              break
        else:
          break
    if(not video_input):
      cv2.destroyAllWindows()
    else:
      out.release()

In [10]:
video('El error de Berto Romero con el Zara.mp4')

Procesando video en: ./Output/El error de Berto Romero con el Zara_out.avi...
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3